In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from scipy.stats import zscore
import seaborn as sns
from sklearn.cross_decomposition import PLSRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from itertools import cycle
import vips
import random
from collections import Counter
from tqdm import tqdm

# Making all figures
 - Test error (PLSR & RF)
 - Clustering (DE genes) -> GO processes (done)
 - PCA (DE genes)
 

In [2]:

## ALREADY PREPROCESSED 
#### Min gene count = 0 for 15% of samples

train_fpkm = pd.read_csv("../data/BAL/train_fpkm_minfilter0_log2.csv",sep='\t',index_col=0)
train_meta = pd.read_csv('../data/BAL/Reseq_ALL/meta_reseq.csv', sep='\t', index_col='sample_id')

test_fpkm = pd.read_csv("../data/BAL/test_fpkm_minfilter0_log2.csv", sep='\t', index_col=0)
test_meta = pd.read_csv('../data/BAL/meta_r0.csv', sep='\t', index_col=0)

In [3]:
print test_fpkm.shape, test_meta.shape
print train_fpkm.shape, train_meta.shape

(32, 26081) (32, 2)
(95, 11957) (95, 32)


In [4]:
def getXy(df, dfm, celltype, norm=False):
    dfm = dfm[dfm['CellType']==celltype]
    df = df.ix[dfm.index]    
    genes = df.columns
    if norm=='zscore': df = df.apply(lambda x: zscore(x))
    X = np.array(df)
    y = dfm['Pneum'].values
    return X, y, genes, df, dfm

In [5]:
def geteven_xy(df, dfm, celltype, sampling='up', norm=False):
    dfm = dfm[dfm['CellType']==celltype]
    y = dfm['Pneum'].values
    df = df.ix[dfm.index]
    #    if z: df = df.apply(lambda x: zscore(x))
    genes = df.columns
    pos = [i for i in range(len(y)) if y[i]==True]
    neg = [i for i in range(len(y)) if y[i]==False]
    
    npos = len(pos)
    nneg = len(neg)

    #print "Num positive: {}".format(len(pos))
    #print "Num negative: {}".format(len(neg))
    if sampling=='down':
        if nneg >= npos:
            neg = random.sample(neg, npos)
        else:
            pos = random.sample(pos, nneg)
    if sampling=='up':
        if nneg >= npos:
            dup_pos = list(np.random.choice(pos, nneg-npos))
            pos = pos + dup_pos ## pos+pos in case not enough in pos
    #            print "duplicated pos:", dup_pos
        else:
            neg = neg + np.random.choice(neg, npos-nneg)

    df = df.ix[(pos+neg),:]
    """
    Normalize from zero to one
    5, 8, 10
    5 - 5 / (10-5)
    8 - 5 / (10-5)
    10 - 5 / (10-5)
    """
    
    if norm=='zero_one': 
        for col in df.columns:
            mx = np.max(df[col])
            mn = np.min(df[col])
            df[col] = df[col].map(lambda x: (x - mn)/(mx - mn))
            
    if norm=='zsc': 
        df = df.apply(lambda x: zscore(x))
    #elif norm==zero_one:
    #    df = df.apply(lambda x: )
    dfm = dfm.ix[(pos+neg),:]
    X = np.array(df)
    y = dfm['Pneum'].values
    print "Num pos after sampling: {}".format(len(pos))
    print "Num neg after sampling: {}".format(len(neg))
    #y = y[pos + neg] 
    #X = X[pos + neg] 
    return X,y, genes, df, dfm

In [23]:
def get_accuracy_stats(celltype, n_top_genes, method='PLSR', 
                       norm=False, random_genes=False, jumble_test=False, jumble_train=False,
                       n_jumbles=20, npcs=5, n_estimators=100, sampling=False):
      
    ### TRAINING
    #################
        
    print "Model = ", celltype
    print "Method = ", method
    #print "N_Top_Genes = ", n_top_genes
    #print "Normalization = ", norm
    
    # Get training data
    X, y, genes, df, dfm = geteven_xy(train_fpkm, train_meta, celltype=celltype, norm=norm, sampling=sampling)
    

    if jumble_train:
        print "---- Entire training set is jumbled ----\n"
        print df.shape
        df = df.apply(np.random.permutation)
        X = np.array(df)

    # Cross-validation (leave-one-out)
    neg_err, pos_err, Q2, neg_accuracy_train, pos_accuracy_train, feature_inds = calc_metrics(X, y, 
                            method=method, n_pcs=npcs, n_estimators=n_estimators, n_top_genes=n_top_genes)

    print "Q Squared: {0:.2f} \n".format(Q2)
    num_neg_correct = sum([(e < 0.5) for e in neg_err])
    num_pos_correct = sum([(e < 0.5) for e in pos_err])
    
    print "\n===== Training ======"
    print "Negative: {0:.3f} ({1}/{2}) ".format(neg_accuracy_train, num_neg_correct, len(neg_err))
    print "Positive: {0:.3f} ({1}/{2}) ".format(pos_accuracy_train, num_pos_correct, len(pos_err))
    
    ## Get most common genes from each LOO model
    all_top_feature_inds = [g for m in feature_inds for g in m]
    c = Counter(all_top_feature_inds)
    top_inds = [i[0] for i in c.most_common(n_top_genes)]
    top_genes_train = [genes[i] for i in top_inds]

    ## SWITCH OUT TOP GENES WITH RANDOM GENES
    if random_genes:
        top_genes_train = np.random.choice(genes, n_top_genes)
        print "\n **Genes for testing are RANDOM** \n"

    ### BUILDING PREDICTIVE MODEL with top genes
    ##############
    X_train = np.array(df[top_genes_train])
    y_train = np.array(dfm['Pneum'])
    
    X_test, y_test, genes_test, df_test, dfm_test = getXy(test_fpkm, 
                    test_meta, celltype=celltype, norm=norm)
    
    X_test = np.array(df_test[top_genes_train])
    
    ### TESTING WITH ACTUAL (non-jumbled) GENES
    ###############
    print "\n===== Testing ====="
    
    if method=='PLSR': 
        model = PLSRegression(5, scale=True)
        
    elif method=='RandomForests': 
        model = RandomForestClassifier(n_estimators=n_estimators)
    else: 
        print "Method {} not recognized".format(method)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_accs = prediction_stats(y_test, y_pred, print_stats=True)
    #print "Neg. test: {0:.3f}".format(test_accs[0])
    #print "Pos. test: {0:.3f}".format(test_accs[1])
    
    if jumble_test:
        X_test = np.array(shuffle(df_test[top_genes_train]).T).T

#     ### JUMBLING GENES
#     ##############
#     all_jumble_accs = []
#     df_train = df[top_genes_train]

#     for i in range(n_jumbles):

#         ## Shuffling the test set for training a model
#         df_jumble = df_train.apply(np.random.permutation)
#         X_jumble = np.array(df_jumble)

#         if method=='PLSR': 
#             model = PLSRegression(5, scale=True)
#         elif method=='RandomForests': 
#             model = RandomForestClassifier(n_estimators=n_estimators)
#         else:
#             print "Method {} not recognized".format(method)

#         model.fit(X_jumble, y_train)
#         y_pred = model.predict(X_test)
        
#         accs = prediction_stats(y_test, y_pred)
#         all_jumble_accs.append(accs)
    
#     jumb_accs = np.array(all_jumble_accs).T
    
#     print "\n===== Jumbled ====="
#     jmean_neg, jstd_neg = np.mean(jumb_accs[0]), np.std(jumb_accs[0])
#     jmean_pos, jstd_pos = np.mean(jumb_accs[1]), np.std(jumb_accs[1])
    
#     print "Neg. mean {0:.3f}, std {1:.3f}".format(jmean_neg, jstd_neg)
#     print "Pos. mean {0:.3f}, std {1:.3f}\n".format(jmean_pos, jstd_pos)
    
#     print "Test # Std. above Jumble"
#     neg_std_above_mean = (test_accs[0] - jmean_neg) / jstd_neg
#     pos_std_above_mean = (test_accs[1] - jmean_pos) / jstd_pos
#     print "Negative: {0:.2f}".format(neg_std_above_mean)
#     print "Positive: {0:.2f}\n".format(pos_std_above_mean)
    
#     print "Negative change {0:.2f}%".format(float(test_accs[0]-np.mean(jumb_accs[0])) / np.mean(jumb_accs[0]) * 100)
#     print "Positive change {0:.2f}%".format(float(test_accs[1]-np.mean(jumb_accs[1])) / np.mean(jumb_accs[0]) * 100)                                             
#     #neg_accuracy_train, neg_accuracy_test, pos_accuracy_train, pos_accuracy_test

In [21]:
def prediction_stats(y_test, y_pred, print_stats=False):
    
    neg_err = [float(abs(y_pred[i]-y_test[i])) for i in range(len(y_pred)) if y_test[i]==0]
    num_neg_correct = sum([(e < 0.5) for e in neg_err])
    neg_accuracy_test = float(num_neg_correct) / len(neg_err)


    pos_err = [float(abs(y_pred[i]-y_test[i])) for i in range(len(y_pred)) if y_test[i]==1]
    num_pos_correct = sum([(e < 0.5) for e in pos_err])
    pos_accuracy_test = float(num_pos_correct) / len(pos_err)
    
    if print_stats:
        print "Negative: {0:.3f} ({1}/{2}) ".format(neg_accuracy_test, num_neg_correct, len(neg_err))        
        print "Positive: {0:.3f} ({1}/{2}) ".format(pos_accuracy_test, num_pos_correct, len(pos_err))

    return neg_accuracy_test, pos_accuracy_test

# Layout for analysis:
 1. Get test accuracies, random=False, sampling=False, jumble_train=False
 2. Is there meaning in the genes? -> change [1] to have jumble_train=True
 3. Is the model based on the top 100 genes better than based on random genes?
 4. Is the model based on the top 100 genes better than the jumbled 100 genes

# 1. Just the results

In [35]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=5, jumble_train=False, #method='RandomForests', 
                       norm='zscore', random_genes=False, jumble_test=False, sampling='')

Model =  AM
Method =  PLSR
Num pos after sampling: 19
Num neg after sampling: 30
Q Squared: -0.23 


===== Training ======
Negative: 0.600 (18/30) 
Positive: 0.632 (12/19) 

===== Testing =====
Negative: 1.000 (14/14) 
Positive: 0.000 (0/6) 


# 2. Jumble all training data

In [36]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=5, jumble_train=True, #method='RandomForests', 
                       norm='zscore', random_genes=False, jumble_test=False, sampling='')

Model =  AM
Method =  PLSR
Num pos after sampling: 19
Num neg after sampling: 30
---- Entire training set is jumbled ----

(49, 11957)
Q Squared: -0.44 


===== Training ======
Negative: 0.567 (17/30) 
Positive: 0.368 (7/19) 

===== Testing =====
Negative: 1.000 (14/14) 
Positive: 0.000 (0/6) 


### For PLSR, jumbled training data leads to same training error as original data

In [241]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, jumble_train=False, #method='RandomForests', 
                       norm='zscore', random_genes=True, jumble_test=False, sampling=False)

Model =  AM
Method =  PLSR
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 19
Num neg after sampling: 30
Q Squared: -0.69 


===== Training ======
Negative: 0.733 (22/30) 
Positive: 0.263 (5/19) 

 **Genes for testing are RANDOM** 


===== Testing =====
Negative: 0.000 (0/14) 
Positive: 0.167 (1/6) 

===== Jumbled =====
Neg. mean 0.021, std 0.033
Pos. mean 0.033, std 0.067

Test # Std. above Jumble
Negative: -0.65
Positive: 2.00

Negative change -100.00%
Positive change 622.22%


### ^^ When using random genes instead of top genes, test accuracy is low

In [243]:
_ = get_accuracy_stats(celltype='CD163', n_top_genes=100, jumble_train=False, method='RandomForests', 
                       norm='zscore', random_genes=False, jumble_test=False, sampling=False)

Model =  CD163
Method =  RandomForests
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 15
Num neg after sampling: 31
Q Squared: -0.09 


===== Training ======
Negative: 0.968 (30/31) 
Positive: 0.333 (5/15) 

===== Testing =====
Negative: 0.000 (0/5) 
Positive: 1.000 (7/7) 

===== Jumbled =====
Neg. mean 0.480, std 0.354
Pos. mean 0.479, std 0.253

Test # Std. above Jumble
Negative: -1.35
Positive: 2.06

Negative change -100.00%
Positive change 108.63%


In [227]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, method='RandomForests', jumble_train=True,
                       norm='zscore', random_genes=False, sampling=False)

Model =  AM
Method =  RandomForests
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 19
Num neg after sampling: 30
Q Squared: -0.89 


===== Training ======
Negative: 0.700 (21/30) 
Positive: 0.316 (6/19) 

===== Testing =====
Negative: 0.286 (4/14) 
Positive: 0.500 (3/6) 

===== Jumbled =====
Neg. mean 0.500, std 0.370
Pos. mean 0.467, std 0.375

Test # Std. above Jumble
Negative: -0.58
Positive: 0.09

Negative change -42.86%
Positive change 6.67%


In [232]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, method='RandomForests', jumble_train=True,
                       norm='zscore', random_genes=False, sampling=False)

Model =  AM
Method =  RandomForests
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 19
Num neg after sampling: 30
---- Entire training set is jumbled ----

Q Squared: -0.98 


===== Training ======
Negative: 0.833 (25/30) 
Positive: 0.053 (1/19) 

===== Testing =====
Negative: 0.286 (4/14) 
Positive: 0.833 (5/6) 

===== Jumbled =====
Neg. mean 0.786, std 0.262
Pos. mean 0.333, std 0.307

Test # Std. above Jumble
Negative: -1.91
Positive: 1.63

Negative change -63.64%
Positive change 63.64%


In [233]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, method='RandomForests', jumble_train=True,
                       norm='zscore', random_genes=False, sampling=False)

Model =  AM
Method =  RandomForests
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 19
Num neg after sampling: 30
---- Entire training set is jumbled ----

Q Squared: -0.89 


===== Training ======
Negative: 0.833 (25/30) 
Positive: 0.105 (2/19) 

===== Testing =====
Negative: 0.429 (6/14) 
Positive: 0.833 (5/6) 

===== Jumbled =====
Neg. mean 0.807, std 0.196
Pos. mean 0.267, std 0.322

Test # Std. above Jumble
Negative: -1.93
Positive: 1.76

Negative change -46.90%
Positive change 70.21%


# PLSR predictions, down-sampling

In [202]:
## When top genes are replaec with random genes, testing and jumbling predictions -> 0

In [201]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, 
                       norm='zscore', random_genes=True, jumble_test=False, sampling='down')

Model =  AM
Method =  PLSR
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 19
Num neg after sampling: 19
Q Squared: -0.62 


===== Training ======
Negative: 0.474 (9/19) 
Positive: 0.368 (7/19) 

 **Genes for testing are RANDOM** 


===== Testing =====
Negative: 0.000 (0/14) 
Positive: 0.000 (0/6) 

===== Jumbled =====
Neg. mean 0.004, std 0.016
Pos. mean 0.000, std 0.000

Improvement (Test) vs. Jumbled
Negative change -100.00%
Positive change 0.00%


In [ ]:
## With the top genes, prediction using jumbled training -> 50/50 (chance but with large deviation)
## Testing -> 100% negative, 0% positive

In [203]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, 
                       norm='zscore', random_genes=False, jumble_test=False, sampling='down')

Model =  AM
Method =  PLSR
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 19
Num neg after sampling: 19
Q Squared: -0.47 


===== Training ======
Negative: 0.632 (12/19) 
Positive: 0.579 (11/19) 

===== Testing =====
Negative: 1.000 (14/14) 
Positive: 0.000 (0/6) 

===== Jumbled =====
Neg. mean 0.439, std 0.456
Pos. mean 0.517, std 0.456

Improvement (Test) vs. Jumbled
Negative change 127.64%
Positive change -117.62%


# PLSR predictions, up-sampling

In [202]:
## When top genes are replaec with random genes, testing and jumbling predictions -> 0

In [204]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, 
                       norm='zscore', random_genes=True, jumble_test=False, sampling='up')

Model =  AM
Method =  PLSR
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 30
Num neg after sampling: 30
Q Squared: 0.11 


===== Training ======
Negative: 0.700 (21/30) 
Positive: 0.767 (23/30) 

 **Genes for testing are RANDOM** 


===== Testing =====
Negative: 0.000 (0/14) 
Positive: 0.000 (0/6) 

===== Jumbled =====
Neg. mean 0.000, std 0.000
Pos. mean 0.000, std 0.000

Improvement (Test) vs. Jumbled
Negative change nan%
Positive change nan%


In [ ]:
## With the top genes, prediction using jumbled training -> 50/50 (chance but with large deviation)
## Testing -> 100% negative, 0% positive

In [205]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, 
                       norm='zscore', random_genes=False, jumble_test=False, sampling='up')

Model =  AM
Method =  PLSR
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 30
Num neg after sampling: 30
Q Squared: -0.06 


===== Training ======
Negative: 0.733 (22/30) 
Positive: 0.733 (22/30) 

===== Testing =====
Negative: 0.786 (11/14) 
Positive: 0.000 (0/6) 

===== Jumbled =====
Neg. mean 0.436, std 0.445
Pos. mean 0.542, std 0.428

Improvement (Test) vs. Jumbled
Negative change 80.33%
Positive change -124.32%


# PLSR predictions, original data

In [202]:
## When top genes are replaec with random genes, testing and jumbling predictions -> 0

In [206]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, 
                       norm='zscore', random_genes=True, jumble_test=False, sampling=False)

Model =  AM
Method =  PLSR
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 19
Num neg after sampling: 30
Q Squared: -0.69 


===== Training ======
Negative: 0.733 (22/30) 
Positive: 0.263 (5/19) 

 **Genes for testing are RANDOM** 


===== Testing =====
Negative: 0.000 (0/14) 
Positive: 0.167 (1/6) 

===== Jumbled =====
Neg. mean 0.007, std 0.021
Pos. mean 0.000, std 0.000

Improvement (Test) vs. Jumbled
Negative change -100.00%
Positive change 2333.33%


In [ ]:
## With the top genes, prediction using jumbled training -> 50/50 (chance but with large deviation)
## Testing -> 100% negative, 0% positive

In [207]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, 
                       norm='zscore', random_genes=False, jumble_test=False, sampling=False)

Model =  AM
Method =  PLSR
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 19
Num neg after sampling: 30
Q Squared: -0.69 


===== Training ======
Negative: 0.733 (22/30) 
Positive: 0.263 (5/19) 

===== Testing =====
Negative: 1.000 (14/14) 
Positive: 0.000 (0/6) 

===== Jumbled =====
Neg. mean 0.486, std 0.463
Pos. mean 0.458, std 0.465

Improvement (Test) vs. Jumbled
Negative change 105.88%
Positive change -94.36%


In [208]:
_ = get_accuracy_stats(celltype='CD163', n_top_genes=100, 
                       norm='zscore', random_genes=False, jumble_test=False)

Model =  CD163
Method =  PLSR
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Num pos after sampling: 15
Num neg after sampling: 31
Q Squared: 0.12 


===== Training ======
Negative: 0.968 (30/31) 
Positive: 0.467 (7/15) 

===== Testing =====
Negative: 0.000 (0/5) 
Positive: 1.000 (7/7) 

===== Jumbled =====
Neg. mean 0.630, std 0.439
Pos. mean 0.243, std 0.391

Improvement (Test) vs. Jumbled
Negative change -100.00%
Positive change 120.18%


In [136]:
_ = get_accuracy_stats(celltype='CD163', n_top_genes=10, 
                       norm='zscore', random_genes=False, jumble_test=False)

Model =  CD163
Method =  PLSR
N_Top_Genes =  10
Normalization =  zscore
Jumble Test =  False
Q Squared: -0.11 


===== Training ======
Negative: 0.903 (28/31) 
Positive: 0.267 (4/15) 

===== Testing =====
Negative: 0.800 (4/5) 
Positive: 0.429 (3/7) 

===== Jumbled =====
Neg. mean 0.780, std 0.166
Pos. mean 0.286, std 0.128

Improvement (Test) vs. Jumbled
Negative change 2.56%
Positive change 18.32%


# PLSR predictions

In [146]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, 
                       norm='zscore', random_genes=False, jumble_test=False)

Model =  AM
Method =  PLSR
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Q Squared: -0.33 


===== Training ======
Negative: 0.700 (21/30) 
Positive: 0.105 (2/19) 

===== Testing =====
Negative: 0.857 (12/14) 
Positive: 0.667 (4/6) 

===== Jumbled =====
Neg. mean 0.736, std 0.109
Pos. mean 0.308, std 0.231

Improvement (Test) vs. Jumbled
Negative change 16.50%
Positive change 48.71%


In [140]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=10, 
                       norm='zscore', random_genes=False, jumble_test=False)

Model =  AM
Method =  PLSR
N_Top_Genes =  10
Normalization =  zscore
Jumble Test =  False
Q Squared: -0.14 


===== Training ======
Negative: 0.733 (22/30) 
Positive: 0.368 (7/19) 

===== Testing =====
Negative: 0.571 (8/14) 
Positive: 0.333 (2/6) 

===== Jumbled =====
Neg. mean 0.754, std 0.094
Pos. mean 0.650, std 0.229

Improvement (Test) vs. Jumbled
Negative change -24.17%
Positive change -42.02%


In [141]:
_ = get_accuracy_stats(celltype='CD163', n_top_genes=100, 
                       norm='zscore', random_genes=False, jumble_test=False)

Model =  CD163
Method =  PLSR
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Q Squared: -0.13 


===== Training ======
Negative: 0.968 (30/31) 
Positive: 0.067 (1/15) 

===== Testing =====
Negative: 1.000 (5/5) 
Positive: 0.286 (2/7) 

===== Jumbled =====
Neg. mean 0.870, std 0.131
Pos. mean 0.157, std 0.149

Improvement (Test) vs. Jumbled
Negative change 14.94%
Positive change 14.78%


In [136]:
_ = get_accuracy_stats(celltype='CD163', n_top_genes=10, 
                       norm='zscore', random_genes=False, jumble_test=False)

Model =  CD163
Method =  PLSR
N_Top_Genes =  10
Normalization =  zscore
Jumble Test =  False
Q Squared: -0.11 


===== Training ======
Negative: 0.903 (28/31) 
Positive: 0.267 (4/15) 

===== Testing =====
Negative: 0.800 (4/5) 
Positive: 0.429 (3/7) 

===== Jumbled =====
Neg. mean 0.780, std 0.166
Pos. mean 0.286, std 0.128

Improvement (Test) vs. Jumbled
Negative change 2.56%
Positive change 18.32%


# RF predictions

In [142]:
dft = get_accuracy_stats(celltype='AM', n_top_genes=100, method='RandomForests',
                       norm='zscore', random_genes=False, jumble_test=False)

Model =  AM
Method =  RandomForests
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Q Squared: -1.15 


===== Training ======
Negative: 0.667 (20/30) 
Positive: 0.211 (4/19) 

===== Testing =====
Negative: 0.857 (12/14) 
Positive: 0.833 (5/6) 

===== Jumbled =====
Neg. mean 0.889, std 0.086
Pos. mean 0.217, std 0.183

Improvement (Test) vs. Jumbled
Negative change -3.61%
Positive change 69.34%


In [143]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=10, method='RandomForests',
                       norm='zscore', random_genes=False, jumble_test=False)

Model =  AM
Method =  RandomForests
N_Top_Genes =  10
Normalization =  zscore
Jumble Test =  False
Q Squared: -0.81 


===== Training ======
Negative: 0.700 (21/30) 
Positive: 0.368 (7/19) 

===== Testing =====
Negative: 0.929 (13/14) 
Positive: 0.500 (3/6) 

===== Jumbled =====
Neg. mean 0.764, std 0.161
Pos. mean 0.183, std 0.217

Improvement (Test) vs. Jumbled
Negative change 21.50%
Positive change 41.43%


In [144]:
_ = get_accuracy_stats(celltype='CD163', n_top_genes=100, method='RandomForests',
                       norm='zscore', random_genes=False, jumble_test=False)

Model =  CD163
Method =  RandomForests
N_Top_Genes =  100
Normalization =  zscore
Jumble Test =  False
Q Squared: -0.09 


===== Training ======
Negative: 0.935 (29/31) 
Positive: 0.400 (6/15) 

===== Testing =====
Negative: 1.000 (5/5) 
Positive: 0.429 (3/7) 

===== Jumbled =====
Neg. mean 0.920, std 0.098
Pos. mean 0.079, std 0.106

Improvement (Test) vs. Jumbled
Negative change 8.70%
Positive change 38.04%


In [145]:
_ = get_accuracy_stats(celltype='CD163', n_top_genes=10, method='RandomForests',
                       norm='zscore', random_genes=False, jumble_test=False)

Model =  CD163
Method =  RandomForests
N_Top_Genes =  10
Normalization =  zscore
Jumble Test =  False
Q Squared: -0.78 


===== Training ======
Negative: 0.742 (23/31) 
Positive: 0.333 (5/15) 

===== Testing =====
Negative: 1.000 (5/5) 
Positive: 0.429 (3/7) 

===== Jumbled =====
Neg. mean 0.900, std 0.118
Pos. mean 0.179, std 0.174

Improvement (Test) vs. Jumbled
Negative change 11.11%
Positive change 27.78%


## AM testing, PLS Vs RF

### 100 genes

In [ ]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=100, norm='zscore', random_genes=False)

In [ ]:
_ = get_accuracy_stats(celltype='AM', method='RandomForests', n_top_genes=100, norm='zscore', random_genes=False)

### 10 genes

In [ ]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=10, norm='zscore', random_genes=False)

In [ ]:
_ = get_accuracy_stats(celltype='AM', method='RandomForests', n_top_genes=10, norm='zscore', random_genes=False)

### 10 genes, random

In [ ]:
_ = get_accuracy_stats(celltype='AM', n_top_genes=10, norm='zscore', random_genes=True)

In [ ]:
_ = get_accuracy_stats(celltype='AM', method='RandomForests', n_top_genes=10, norm='zscore', random_genes=True)

## Calc metrics()

## Train clustering

In [ ]:
plot_cluster(df_train_top, dfm_train)

## Test clustering

In [ ]:
plot_cluster(df_test_top, dfm_test)

# AM predictions, clustering

In [ ]:
top_genes_train, df_train_top, dfm_train, df_test_top, dfm_test = get_accuracy_stats(celltype='AM', n_top_genes=120, norm='zsc', random_genes=False)

In [ ]:
plot_cluster(df_test_top, dfm_test)

In [ ]:
plot_cluster(df_train_top, dfm_train)

## +++++

In [ ]:
def plot_cluster(df, dfm):
    
    dfz = df.apply(zscore, axis=1)
    dfz = dfz.apply(zscore, axis=0)
    dfplot = dfz.T

    dfplot = dfplot.rename(columns={c: dfm.loc[c,'Pneum'] for c in dfplot.columns})

    dfplot.columns
    fig = sns.clustermap(dfplot, col_cluster=True)

In [ ]:
plot_cluster(df_test_top, dfm_test)

In [ ]:
plot_cluster(df_train_top, dfm_train)

In [ ]:
#determine the best number of genes for the training model
def plotQ2(r=np.arange(10,205,5), celltype=False, z=True):
    Q2s = []
    for n_top_genes in tqdm(r):
        X, y, genes, df, dfm = geteven_xy(train_fpkm, meta, celltype=celltype, norm='zsc')
        neg_err, pos_err, Q2, neg_corr, pos_corr, vip_inds = calc_metrics(X, y, 
                                                        n_pcs=5, n_top_genes=n_top_genes)
        Q2s.append(Q2)
    plt.plot(r, Q2s)
    return None #Q2s

In [ ]:
plotQ2(r=np.arange(10,155,10), celltype='AM')

In [ ]:
plotQ2(r=np.arange(10,155,5), celltype='CD163')

In [10]:
def vipp(x, y, t, w):

    """
    From original MATLAB code
    See https://code.google.com/p/carspls/

    #+++ vip=vipp(x,y,t,w);
    #+++ t: scores, which can be obtained by pls_nipals.m
    #+++ w: weight, which can be obtained by pls_nipals.m
    #+++ to calculate the vip for each variable to the response;
    #+++ vip=sqrt(p*q/s);
    """
    #initializing
    [p, h] = w.shape
    co = np.matrix(np.zeros([1, h]))

    # Calculate s
    for ii in range(h):
        corr = np.corrcoef(y, t[:, ii], rowvar=0)
        co[0, ii] = corr[0, 1]**2
    s = np.sum(co)

    # Calculate q
    # This has been linearized to replace the original nested for loop
    w_power = np.power(w, 2)
    d = np.multiply(w_power, co)
    q = np.sum(d, 1)
    vip = np.sqrt(p*q/s)
    return vip

In [9]:
def calc_metrics(X, y, n_pcs, n_estimators, n_top_genes, method='PLSR'):
    
    n_samples = len(X)
    pred = []
    feat_inds_all = []
    #print X.shape, y.shape
    
    ### Leave-One-Out Cross-validation
    
    for sample in range(n_samples):
        samples = range(n_samples)
        samples.remove(sample)
        X_train = X[(samples)]
        y_train = y[(samples)]
        #print X_t.shape    
        
        if method=='PLSR':### Get VIPs for model built without the LOO sample
            plsv = PLSRegression(n_pcs, scale=False)
            plsv.fit(X_train, y_train)
            vips = vipp(X_train, y_train, plsv.x_scores_, plsv.x_weights_)
            vips = [float(v) for v in vips]
            vip_inds = np.argsort(vips)[::-1][:n_top_genes]
            feat_inds_all.append(vip_inds)

            ### Xn takes the original X to get the loo sample
            X_top_features = X.T[vip_inds].T
            loo_sample = np.array(X_top_features[sample]).reshape((1,-1))

            ### Filter the samples 
            X_train_top_features = X_train.T[vip_inds].T
            pls = PLSRegression(n_pcs, scale=False)
            pls.fit(X_train_top_features, y_train)
            pred.append(float(pls.predict(loo_sample)))

        
        elif method=='RandomForests':        ### Get VIPs for model built without the LOO sample
            rfc = RandomForestClassifier(n_estimators=n_estimators)
            rfc.fit(X_train, y_train)
            feat_imp = rfc.feature_importances_
            feat_inds = np.argsort(feat_imp)[::-1][:n_top_genes]
            feat_inds_all.append(feat_inds)

            ### Xn takes the original X to get the loo sample
            X_top_features = X.T[feat_inds].T
            loo_sample = np.array(X_top_features[sample]).reshape((1,-1))

            ### Filter the samples 
            X_train_top_features = X_train.T[feat_inds].T
            rfc = RandomForestClassifier(n_estimators=n_estimators)
            rfc.fit(X_train_top_features, y_train)

            #print "prediction: ", float(rfc.predict(loo_sample))
            pred.append(float(rfc.predict(loo_sample)))
        
        else:
            print "Method {} not found".format(method)
            

    # Calculate metrics
    ### Q squared
    num = sum([float((pred[i] - y[i]))**2 for i in range(len(pred))])
    den = sum([(y[i] - np.mean(y))**2 for i in range(len(pred))])
    Q2 = float(1 - num/den)

    ### Prediction error
    errs = [abs(float((pred[i] - y[i]))) for i in range(len(pred))]
 
    ### Percent correct for neg and pos Pneumonia
    neg_err = [errs[i] for i in range(len(errs)) if y[i]==0]  
    neg_corr = [e < 0.5 for e in neg_err]   
    neg_corr = float(sum(neg_corr)) / len(neg_err)

    
    pos_err = [errs[i] for i in range(len(errs)) if y[i]==1]
    pos_corr = [e < 0.5 for e in pos_err]
    pos_corr = float(sum(pos_corr)) / len(pos_err)
    

    #print "num: {0: .3f}, den: {1: .3f}, 
    #print "Q^2: {0: .3f}".format(Q2)
    
    return neg_err, pos_err, Q2, neg_corr, pos_corr, feat_inds_all
    #return [float(abs(pred[p] - y[p])) for p in range(len(pred))]

In [ ]:
def calc_Q2_kftest(X, y, k=5, n_rand=10, n_pcs=5, method='PLSR'):
    n_samples = len(X)
    pred = []
    Q2s = []
    for ki in range(n_rand):
        sample_ind = range(n_samples)
        withheld_samples = random.sample(sample_ind, k)
        lo_samples = X[withheld_samples]
        for s in withheld_samples:
            sample_ind.remove(s)
        X_t = X[(sample_ind)]
        y_t = y[(sample_ind)]
        if method=='PLSR':
            model = PLSRegression(n_pcs, scale=False)
        elif method=='RF':
            model = RandomForestClassifier()
        elif method=='SVM':
            model = SVC()
        else:
            print "Method not found"
        model.fit(X_t, y_t)

        for s in withheld_samples:
            pred.append(float(model.predict(X[s])))
        #Eprint "samples", withheld_samples
        #Eprint "pred", pred
        num = sum([(pred[i] - y[s])**2 for i, s in enumerate(withheld_samples)])
        den = sum([(y[s] - np.mean(y[withheld_samples]))**2 for s in withheld_samples])
        Q2 = float(1- num/den)
        Q2s.append(Q2)
    print "Q^2: mean={0: .2f}, std={1: .2f}".format(np.mean(Q2s),np.std(Q2s))
    #return [float(abs(pred[p] - y[p])) for p in range(len(pred))]